In [40]:
# ใช้สำหรับการ import data from binance
import ccxt
import pandas as pd

# สร้างเชื่อมต่อกับ Binance
exchange = ccxt.binance()
symbol = 'ETH/USD'  # คู่เหรียญ
timeframe = '1d'  # กรอบเวลา: 1 วัน

In [41]:
# ดึงข้อมูลย้อนหลัง 
data = exchange.fetch_ohlcv(symbol, timeframe, limit=300)  # ล่าสุด 300 แท่ง
df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']) #ปรับเพื่อแก้ว่าต้องการข้อมูลอะไรเพิ่ม
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

from ta.momentum import RSIIndicator
from ta.trend import MACD

# คำนวณ RSI
rsi = RSIIndicator(close=df['close'], window=14)
df['rsi'] = rsi.rsi()

# คำนวณ MACD
macd = MACD(close=df['close'])
df['macd'] = macd.macd()  # ค่า MACD Line
df['macd_signal'] = macd.macd_signal()  # ค่า Signal Line
df['macd_diff'] = macd.macd_diff()  # ค่า MACD different

# Drop missing value from the dataset
df.dropna(inplace=True)

#สร้าง Target Variable ถ้าราคาปิดของอีกวันสูงขึ้นจะเป็นการ บ่งบอกว่า Good Target

df['target'] = (df['close'].shift(-1) > df['close']).astype(int)

# การเปลี่ยนแปลงของราคา
df['price_change'] = df['close'] - df['open']

# splitting data สำหรับ test แบบ timeframe series
# แปลง timestamp เป็น datetime format (หากยังไม่ได้แปลง)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# แยกข้อมูล Train (เดือน 10, 11, 12 ปี 2024)
train_data = df[(df['timestamp'] >= '2024-10-01') & (df['timestamp'] <= '2024-12-31')]

# แยกข้อมูล Test (เดือน 1 ปี 2025)
test_data = df[(df['timestamp'] >= '2025-01-01') & (df['timestamp'] <= '2025-01-31')]

# ทำ Standardization (Z-Score Scaling) data ก่อน fitting model เพระาหน่วยแตกต่างกัน
# ปรับข้อมูลให้อยู่ในรูปค่าเฉลี่ย 0 และส่วนเบี่ยงเบนมาตรฐาน 1
from sklearn.preprocessing import StandardScaler

# เลือก Features ที่ต้องการ Normalize
features_to_scale = ['rsi', 'macd_diff', 'price_change', 'volume', 'close']
target_variable = 'target'

# สร้าง Scaler
scaler = StandardScaler()

# Fit และ Transform ข้อมูล
train_data.loc[:, features_to_scale] = scaler.fit_transform(train_data[features_to_scale])
test_data.loc[:, features_to_scale] = scaler.transform(test_data[features_to_scale])

train_data = train_data.loc[:, features_to_scale + [target_variable]]

# ระบุ Feature และ Target
features_to_scale = ['rsi', 'macd_diff', 'price_change', 'volume', 'close']
target_variable = 'target'  # เป้าหมายที่ต้องการพยากรณ์

# แยก X (Features) และ y (Target)
x_train = train_data[features_to_scale]
y_train = train_data[target_variable]

x_test = test_data[features_to_scale]
y_test = test_data[target_variable]

In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

# สร้างและฝึก Random Forest
rf_model = RandomForestClassifier(random_state=42, class_weight={0: 1, 1: 3})
rf_model.fit(x_train, y_train)

# ดึงค่า Feature Importance
rf_feature_importance = rf_model.feature_importances_
rf_features = x_train.columns  

# แสดงผล Feature Importances พร้อมชื่อ Feature
rf_importance_df = pd.DataFrame({'Feature': rf_features, 'Importance': rf_feature_importance})
rf_importance_df = rf_importance_df.sort_values(by='Importance', ascending=False)
print("Feature Importances (Random Forest):")
print(rf_importance_df)

# ทำนายผล
# Predict with adjusted threshold
y_pred_proba = rf_model.predict_proba(x_test)[:, 1]
y_pred_adjusted = (y_pred_proba >= 0.5).astype(int)

# ประเมินผล
print("Classification Report (Random Forest):")
print(classification_report(y_test, y_pred_adjusted))


Feature Importances (Random Forest):
        Feature  Importance
2  price_change    0.242847
1     macd_diff    0.206088
3        volume    0.203712
4         close    0.191267
0           rsi    0.156086
Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.58      0.47      0.52        15
           1       0.56      0.67      0.61        15

    accuracy                           0.57        30
   macro avg       0.57      0.57      0.56        30
weighted avg       0.57      0.57      0.56        30



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': [None, {0: 1, 1: 2}, {0: 1, 1: 3}]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, scoring='precision', cv=10)
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

In [ ]:
from sklearn.metrics import classification_report

# ทำนายผลด้วยโมเดลที่ดีที่สุด
y_pred_proba = best_model.predict_proba(x_test)[:, 1]  # ใช้ predict_proba เพื่อปรับ threshold
y_pred_adjusted = (y_pred_proba >= 0.6).astype(int)  # สามารถปรับ threshold ได้ เช่น 0.6

# แสดงผลลัพธ์
print("Classification Report (Best Model):")
print(classification_report(y_test, y_pred_adjusted))

In [ ]:
# Extract the last row of features from the scaled last 7 days data
input_data = last_7_days.iloc[-1:][features_to_scale]

# Ensure the input is a DataFrame with feature names for StandardScaler
input_data_scaled = scaler.transform(input_data)

# Convert back to a DataFrame to preserve feature names
input_data_scaled_df = pd.DataFrame(input_data_scaled, columns=features_to_scale)

# Predict using the Random Forest model
prediction = rf_model.predict(input_data_scaled_df)

# Display prediction
print("Prediction:", prediction)